# Fase 6




## Resumen del flujo recomendado antes del modelado RNN

### 1. Limpieza estructural
- Eliminar columnas con más de 30–40% de valores nulos.  
- Eliminar columnas constantes o duplicadas.  
- Imputar los valores faltantes restantes (`forward-fill`, media o mediana según el caso).  
- Escalar todas las variables (`StandardScaler` o `RobustScaler`).

---

### 2. Transformaciones de mercado
- Convertir precios a **retornos logarítmicos**:  
  \[
  r_t = \log\left(\frac{P_t}{P_{t-1}}\right)
  \]
- Calcular **volatilidades móviles** (rolling std) de 30 y 90 días para capturar dinámica temporal.

---

### 3. Selección de variables relevantes
- Calcular la **correlación en valor absoluto** de todas las variables con los **retornos de BBVA y Santander**.  
- Seleccionar las **Top N** (por ejemplo 50–80) variables más correlacionadas.  
- Este subconjunto representa las features más informativas para el modelo.

---

### 4. Reducción de dimensionalidad (PCA)
- Aplicar **PCA** sobre las variables restantes (no seleccionadas en el paso anterior).  
- Conservar las componentes que expliquen entre **90–95% de la varianza total** (≈10–30 componentes).  
- Combinar:
  - Las variables seleccionadas por correlación.  
  - Las componentes principales del PCA.

📦 **Resultado:**  
Un dataset limpio, compacto y optimizado con unas **~60 variables finales**, que combinan la relevancia estadística (correlación) y la representación comprimida (PCA), ideal para la siguiente fase de modelado RNN.


## Libreries


In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

## 1. Limpieza

In [3]:
# ============================
# CONFIG
# ============================
DATA_PATH = Path("../../data/processed/data.csv.gz")

# ============================
# CARGA
# ============================
data = pd.read_csv(
    DATA_PATH,
    compression="gzip",
    parse_dates=["Date"],
    index_col="Date"
)
print(f"Dataset cargado con {data.shape[0]} filas y {data.shape[1]} columnas.")
display(data.head())

# ============================
# 1️⃣ CONVERTIR SOLO COLUMNAS NUMÉRICAS
# ============================
num_cols = data.select_dtypes(include=['int64','float64','object']).columns
for col in num_cols:
    data[col] = pd.to_numeric(data[col], errors='coerce')


Dataset cargado con 44630 filas y 305 columnas.


C:\Users\aleja\AppData\Local\Temp\ipykernel_4756\3744722356.py:9: DtypeWarning: Columns (304) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(


,BKT.MC_Close,BKT.MC_High,BKT.MC_Low,BKT.MC_Open,BKT.MC_Volume,BBVA.MC_Close,BBVA.MC_High,BBVA.MC_Low,BBVA.MC_Open,BBVA.MC_Volume,...,MSCI_Financials.csv_Low,MSCI_Financials.csv_Open,MSCI_Financials.csv_Volume,VIX.csv_Close,VIX.csv_High,VIX.csv_Low,VIX.csv_Open,VIX.csv_Volume,Event_Name,Impact_Score
Date,,,,,,,,,,,,,,,,,,,,,
1913-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# ============================
# 2️⃣ Eliminación de columnas duplicadas exactas
# ============================
data = data.loc[:, ~data.columns.duplicated()]
print(f"Después de eliminar columnas duplicadas, el dataset tiene {data.shape[1]} columnas.")

Después de eliminar columnas duplicadas, el dataset tiene 305 columnas.


In [5]:
# ============================
# 3️⃣ Eliminación de columnas altamente correlacionadas (>0.98) usando NumPy
# ============================
# Convertir a numpy array
arr = data.to_numpy()
# Calcular correlación
corr_matrix = np.corrcoef(arr, rowvar=False)
# Mantener la matriz superior triangular
upper_tri = np.triu(corr_matrix, k=1)

# Columnas a eliminar
to_drop = [data.columns[i] for i in range(len(data.columns)) if any(upper_tri[:, i] > 0.98)]
print(f"Columnas eliminadas por alta correlación: {len(to_drop)}")

# Eliminar columnas
data = data.drop(columns=to_drop)

Columnas eliminadas por alta correlación: 0


In [6]:
# ============================
# 4️⃣ Comprobaciones finales
# ============================
print(f"Dataset limpio → {data.shape[0]} filas y {data.shape[1]} columnas.")
print(f"Nulos restantes: {data.isna().sum().sum()}")
display(data.head())

Dataset limpio → 44630 filas y 305 columnas.
Nulos restantes: 11573266


,BKT.MC_Close,BKT.MC_High,BKT.MC_Low,BKT.MC_Open,BKT.MC_Volume,BBVA.MC_Close,BBVA.MC_High,BBVA.MC_Low,BBVA.MC_Open,BBVA.MC_Volume,...,MSCI_Financials.csv_Low,MSCI_Financials.csv_Open,MSCI_Financials.csv_Volume,VIX.csv_Close,VIX.csv_High,VIX.csv_Low,VIX.csv_Open,VIX.csv_Volume,Event_Name,Impact_Score
Date,,,,,,,,,,,,,,,,,,,,,
1913-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# ============================
# 5️⃣ Guardar dataset limpio temporal
# ============================
output_path = Path("../../data/processed/data_clean.csv.gz")
data.to_csv(output_path, compression="gzip")
print(f"✅ Dataset limpio guardado en: {output_path}")

✅ Dataset limpio guardado en: ..\..\data\processed\data_clean.csv.gz


## 2. Selección de variables relevantes 

**Objetivo:** reducir las ~300 columnas a un set manejable y relevante para la RNN.  
Se hará en dos pasos:

1. **Top correlacionadas:** identificar las variables más correlacionadas (en valor absoluto) con los retornos de BBVA y Santander y seleccionarlas directamente.  
2. **PCA:** aplicar reducción de dimensionalidad al resto de variables para capturar la mayor parte de la varianza con un número reducido de componentes.  


In [8]:
# Cargar dataset limpio
data_path = "../../data/processed/data_clean.csv.gz"
data = pd.read_csv(data_path, parse_dates=['Date'], index_col='Date')

# Seleccionar columnas de precios para crear retornos log
price_cols = ['BBVA.MC_Close', 'SAN.MC_Close']
returns = np.log(data[price_cols] / data[price_cols].shift(1)).dropna()

In [9]:
# Calcular correlación absoluta de todas las columnas con los retornos
corr_df = data.corr().abs()  # correlación de Pearson

# Solo nos interesa correlación con BBVA y SANT
corr_bbva = corr_df['BBVA.MC_Close'].sort_values(ascending=False)
corr_sant = corr_df['SAN.MC_Close'].sort_values(ascending=False)

In [10]:
# Seleccionar top N correlacionadas (ejemplo: top 50)
top_n = 50
top_bbva = corr_bbva.index[1:top_n+1].tolist()  # excluyendo BBVA_Close
top_sant = corr_sant.index[1:top_n+1].tolist()  # excluyendo SANT_Close

# Combinar top variables
top_features = list(set(top_bbva + top_sant))
print(f"Total features top correlacionadas: {len(top_features)}")
print(top_features[:50])  # mostrar primeras 10 para revisión

Total features top correlacionadas: 68
['SAN.MC_Close', 'BKT.MC_Low', 'STOXX50E_Low', 'STOXX50E_Open', 'HSBC_Low', 'UNI.MC_Open', 'HSBC_Open', 'EURJPY.csv_Close', 'ECB_M3_SKK', 'IBEX_High', 'BBVA.MC_Low', 'MSCI_Financials.csv_High', 'IBEX_Close', 'SAB.MC_Low', 'UNI.MC_Low', 'EURJPY.csv_Open', 'UNI.MC_High', 'GDAXI_Close', 'URTH_Low', 'URTH_Close', 'GDAXI_Open', 'STOXX50E_Close', 'STOXX50E_High', 'BBVA.MC_Open', 'BKT.MC_Open', 'N225_Open', 'ECB_M3_HKD', 'FTSE_Close', 'EURJPY.csv_Low', 'FTSE_Low', 'UNI.MC_Close', 'JPM_Open', 'IBEX_Low', 'BKT.MC_Close', 'URTH_Open', 'HSBC_Close', 'ECB_M3_USD', 'GDAXI_Low', 'GDAXI_High', 'CABK.MC_Close', 'JPM_Close', 'SAB.MC_Close', 'SAB.MC_High', 'JPM_High', 'SAN.MC_Low', 'MSCI_Financials.csv_Open', 'FTSE_Open', 'BBVA.MC_Close', 'ECB_M3_CZK', 'FTSE_High']


## 3. PCA para el resto de variables

**Objetivo:** reducir dimensionalidad de las columnas que **no están entre las top 50 correlacionadas** con BBVA y Santander.  
Esto permitirá:

- Mantener la mayor parte de la información de las ~250 columnas restantes.  
- Generar un set compacto y manejable para la RNN.  
- Combinar estas componentes con las top 50 variables para formar el dataset final.

**Pasos:**
1. Separar las columnas top 50 de las restantes.  
2. Escalar las restantes con `StandardScaler`.  
3. Aplicar PCA y conservar suficientes componentes que expliquen ~90–95% de la varianza.  
4. Combinar PCA + top 50 y guardar lista de features finales (`final_features_list.csv`).


In [11]:
# Separar variables restantes (no top 50)
remaining_features = [col for col in data.columns if col not in top_features]

# Escalado
scaler = StandardScaler()
X_scaled = pd.DataFrame(
    scaler.fit_transform(data[remaining_features].fillna(0)),  # rellenar NaN con 0 temporalmente para el scaler
    index=data.index,
    columns=remaining_features
)

# Forward-fill y backward-fill para mantener integridad temporal
X_scaled = X_scaled.fillna(method='ffill').fillna(method='bfill')

# PCA
pca = PCA(n_components=0.95)  # conservar 95% de la varianza
X_pca = pca.fit_transform(X_scaled)

print(f"Número de componentes PCA seleccionadas: {X_pca.shape[1]}")

# Crear DataFrame de componentes PCA
pca_cols = [f'PCA_{i+1}' for i in range(X_pca.shape[1])]
pca_df = pd.DataFrame(X_pca, index=data.index, columns=pca_cols)

C:\Users\aleja\AppData\Local\Temp\ipykernel_4756\3829074476.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_scaled = X_scaled.fillna(method='ffill').fillna(method='bfill')


Número de componentes PCA seleccionadas: 22


In [16]:
# Combinar top_features + PCA
final_data = pd.concat([data[top_features], pca_df], axis=1)

# Lista de features finales
final_features = list(final_data.columns)

# Mostrar columnas y primeras filas
print(final_features)
display(final_data)

# Si Date estaba en el índice, lo convertimos a columna
if 'Date' not in final_data.columns and final_data.index.name == 'Date':
    final_data = final_data.reset_index()

# Guardar DataFrame completo
final_data.to_csv('../../data/processed/final_data.csv.gz', index=False, compression='gzip')
print("DataFrame final guardado en ../../data/processed/final_data.csv.gz.")

# Guardar lista de columnas
final_features = list(final_data.columns)
pd.Series(final_features).to_csv('../../data/processed/final_features_list.csv.gz', index=False)
print("Lista de features guardada en ../../data/processed/final_features_list.csv.gz.")

['SAN.MC_Close', 'BKT.MC_Low', 'STOXX50E_Low', 'STOXX50E_Open', 'HSBC_Low', 'UNI.MC_Open', 'HSBC_Open', 'EURJPY.csv_Close', 'ECB_M3_SKK', 'IBEX_High', 'BBVA.MC_Low', 'MSCI_Financials.csv_High', 'IBEX_Close', 'SAB.MC_Low', 'UNI.MC_Low', 'EURJPY.csv_Open', 'UNI.MC_High', 'GDAXI_Close', 'URTH_Low', 'URTH_Close', 'GDAXI_Open', 'STOXX50E_Close', 'STOXX50E_High', 'BBVA.MC_Open', 'BKT.MC_Open', 'N225_Open', 'ECB_M3_HKD', 'FTSE_Close', 'EURJPY.csv_Low', 'FTSE_Low', 'UNI.MC_Close', 'JPM_Open', 'IBEX_Low', 'BKT.MC_Close', 'URTH_Open', 'HSBC_Close', 'ECB_M3_USD', 'GDAXI_Low', 'GDAXI_High', 'CABK.MC_Close', 'JPM_Close', 'SAB.MC_Close', 'SAB.MC_High', 'JPM_High', 'SAN.MC_Low', 'MSCI_Financials.csv_Open', 'FTSE_Open', 'BBVA.MC_Close', 'ECB_M3_CZK', 'FTSE_High', 'BBVA.MC_High', 'BKT.MC_High', 'EURJPY.csv_High', 'N225_High', 'SAN.MC_Open', 'MSCI_Financials.csv_Close', 'MSCI_Financials.csv_Low', 'N225_Close', 'URTH_High', 'IBEX_Open', 'ECB_M3_JPY', 'CABK.MC_High', 'CABK.MC_Open', 'SAB.MC_Open', 'HSBC_H

,SAN.MC_Close,BKT.MC_Low,STOXX50E_Low,STOXX50E_Open,HSBC_Low,UNI.MC_Open,HSBC_Open,EURJPY.csv_Close,ECB_M3_SKK,IBEX_High,...,PCA_13,PCA_14,PCA_15,PCA_16,PCA_17,PCA_18,PCA_19,PCA_20,PCA_21,PCA_22
Date,,,,,,,,,,,,,,,,,,,,,
1913-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.168185,-0.094069,0.133188,0.159825,-0.137894,-0.004433,-0.204037,-0.060170,0.133580,-0.002773
1913-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.168185,-0.094069,0.133188,0.159825,-0.137894,-0.004433,-0.204037,-0.060170,0.133580,-0.002773
1913-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.168185,-0.094069,0.133188,0.159825,-0.137894,-0.004433,-0.204037,-0.060170,0.133580,-0.002773
1913-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.168185,-0.094069,0.133188,0.159825,-0.137894,-0.004433,-0.204037,-0.060170,0.133580,-0.002773
1913-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.168185,-0.094069,0.133188,0.159825,-0.137894,-0.004433,-0.204037,-0.060170,0.133580,-0.002773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-10-29,8.865,12.85,5691.819824,5701.240234,70.160004,2.350,70.250000,176.867004,NaN,16210.500000,...,-1.417439,0.270332,-0.035416,-3.233211,3.800043,-1.273700,1.316176,-4.846464,-0.938750,-0.844075
2025-10-30,8.750,12.95,5667.100098,5702.770020,69.800003,2.362,69.800003,177.199997,NaN,16117.700195,...,-0.837936,0.501435,0.858817,-5.682608,0.919306,0.624927,0.554233,-4.628295,-0.080748,-0.642446
2025-10-30,8.750,12.95,5667.100098,5702.770020,69.800003,2.362,69.800003,177.199997,NaN,16117.700195,...,-0.739295,0.549786,0.778913,-5.656696,0.947402,0.721576,0.793313,-4.729327,-0.076427,-0.544759


DataFrame final guardado en ../../data/processed/final_data.csv.gz.
Lista de features guardada en ../../data/processed/final_features_list.csv.gz.
